# Regression Evaluation

Notebook for testing regression using the California Housing dataset


# Setup


In [ ]:
import numpy as np
import pandas as pd
import scipy

# This is a draft---don't overengineer!
# NO renaming!
# NO refactoring!
# TODO: Remove this when the draft is done.

In [ ]:
from sklearn.datasets import fetch_california_housing

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Data


In [ ]:
dataset = fetch_california_housing()

In [ ]:
X = pd.DataFrame(dataset["data"], columns=dataset["feature_names"])
X

In [ ]:
y = pd.DataFrame(dataset["target"], columns=dataset["target_names"])
y

In [ ]:
df = pd.concat([X, y], axis=1)

## EDA


In [ ]:
g = sns.PairGrid(df)
g.map_diag(sns.histplot, bins=32)
g.map_offdiag(sns.histplot, bins=32)